# Práctica 3: Representación del conocimiento

__Fecha de entrega: 8 de mayo de 2022__

El objetivo de esta práctica es aplicar los conceptos teóricos vistos en clase en el módulo de Representación de conocimiento. La práctica consta de un único notebook que se entregará en la tarea de entrega habilitada en el Campus  Virtual.

Cada consulta debe contener breves comentarios que expliquen cada tripleta. __No se valorarán consultas sin explicaciones__.

__Número de grupo: 11__

__Nombres de los estudiantes: Aitor Esteban y Edurne Ruiz__

## Consultas SPARQL sobre Wikidata.

En esta práctica vamos a usar el [punto de acceso SPARQL](https://query.wikidata.org/) de Wikidata para contestar las preguntas que se formulan a continuación. Cada pregunta debe ser respondida realizando una única consulta SPARQL. Para cada una de las entidades recuperadas se mostrará __tanto su identificador como su etiqueta__ (nombre de la entidad en lenguaje natural). 

Para cada una de las preguntas debes mostrar tanto la consulta como la respuesta obtenida. Si lo consideras necesario, puedes añadir celdas adicionales en formato _Markdown_ para explicar decisiones que hayas tomado al crear la consulta o cualquier otro dato que consideres interesante.
 
Para resolver estas consultas puedes usar __dos recursos que te recomendamos consultar__ son:

- [Este tutorial de SPARQL](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial).
- [Esta recopilación de ejemplos](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples)

In [2]:
# Asumismos instalado wdsparql
%load_ext autoreload
%load_ext wdsparql
%autoreload 2

### Ejemplo

Recuperar todas las instancias directas de la clase [Cabra (Q2934)](https://www.wikidata.org/wiki/Q2934) que aparecen en la base de conocimiento.

In [ ]:
%%wdsparql
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q2934.  # instancias directas de la clase Cabra
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,item,itemLabel
0,http://www.wikidata.org/entity/Q151345,Billygoat Hennes
1,http://www.wikidata.org/entity/Q3569037,William Windsor
2,http://www.wikidata.org/entity/Q23003932,His Whiskers
3,http://www.wikidata.org/entity/Q24287064,Taffy
4,http://www.wikidata.org/entity/Q41239734,Lance Corporal Shenkin III
5,http://www.wikidata.org/entity/Q41240892,Lance Corporal Shenkin II
6,http://www.wikidata.org/entity/Q41241416,Lance Corporal Shenkin I


### Consulta 1

[Isaac Asimov (Q34981)](https://www.wikidata.org/wiki/Q34981) fue un escritor y profesor de bioquímica en la Universidad de Boston conocido por ser un prolífico autor de obras de ciencia ficción, historia y divulgación científica. 

Vamos a comenzar por averiguar su fecha y lugar de nacimiento (localidad y país al que pertenece la localidad en la actualidad).

Seleccionamos las variables ?birth_date, ?birth_location y ?birth_country para recuperar y utillizamos los códigos de las propiedades para formar tripletas con el código de Asimov como sujeto ?birth_date y ?birth_location como objetos. Ademas, queremos recuperar el país al que pertenece ?birth_location asique lo ponemos de sujeto en la última tripleta para recuperar ?birth_country con la propiedad. Utilizamos ?birth_locationLabel  ?birth_countryLabel para obtener las etiquetas de los códigos en Wikidata.

In [6]:
%%wdsparql

SELECT ?birth_date ?birth_location ?birth_country ?birth_locationLabel  ?birth_countryLabel
WHERE 
{
  wd:Q34981 wdt:P569 ?birth_date.  #fecha de nacimiento de Isaac Asimov (la propiedad date of birth tiene codigo P569)
  wd:Q34981 wdt:P19 ?birth_location.  #localidad de nacimiento de Isaac Asimov (la propiedad place of birth tiene codigo P19 y es el lugar más específico)
  ?birth_location wdt:P17 ?birth_country #pais al que pertenece la localidad de nacimiento de Asimov
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}

,birth_date,birth_location,birth_locationLabel,birth_country,birth_countryLabel
0,1920-01-02T00:00:00Z,http://www.wikidata.org/entity/Q1010621,Petrovichi,http://www.wikidata.org/entity/Q159,Russia


### Consulta 2

A continuación vamos a averiguar todas las distintas profesiones (ocupaciones) que se le reconocen en la base de conocimiento. Queremos obtener los resultados ordenados alfabéticamente por el nombre de la profesión.

Seleccionamos la variable ?ocupacion y utilizamos el codigo de la propiedad ocupacion para formar una tripleta con Asimov como sujeto y ?ocupacion como objeto para que nos devuelva los códigos en Wikidata de sus ocupaciones. Utilizamos ORDER BY para ordenar ascendentemente sus ocupaciones por ?ocupacionLabel, es decir, por las etiquetas de sus ocupaciones.

In [8]:
%%wdsparql
SELECT ?ocupacion ?ocupacionLabel
WHERE 
{
  wd:Q34981 wdt:P106 ?ocupacion.  #ocupaciones de Isaac Asimov (la propiedad ocupacion tiene codigo P106)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
ORDER BY ASC(?ocupacionLabel)

,ocupacion,ocupacionLabel
0,http://www.wikidata.org/entity/Q18814623,autobiographer
1,http://www.wikidata.org/entity/Q2919046,biochemist
2,http://www.wikidata.org/entity/Q1930187,journalist
3,http://www.wikidata.org/entity/Q15980158,non-fiction writer
4,http://www.wikidata.org/entity/Q6625963,novelist
5,http://www.wikidata.org/entity/Q12144794,prosaist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q3745071,science writer
8,http://www.wikidata.org/entity/Q901,scientist
9,http://www.wikidata.org/entity/Q28389,screenwriter


### Consulta 3

De todas esas profesiones, ¿cuáles corresponden con subtipos de [Escritor (Q36180)](https://www.wikidata.org/wiki/Q36180)? Ten en cuenta que la jerarquía de tipos de escritores puede ser muy sofisticada.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Property_paths) te resulte útil.

La segunda tripleta de la consulta obtiene todos los subtipos de Escritor(Q36180) dentro de las ocupaciones seleccionadas de Asimov como en la consulta anterior. Utilizamos * en P279 para "propagar" la propiedad de ser subtipo. Observamos que de esta forma todas las ocupaciones de Asimov son subtipo de Escritor.

In [12]:
%%wdsparql
SELECT ?ocupacion ?ocupacionLabel WHERE {
    wd:Q34981 wdt:P106 ?ocupacion.
    ?ocupacion wdt:P279* wd:Q36180.
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?ocupacionLabel)


,ocupacion,ocupacionLabel
0,http://www.wikidata.org/entity/Q18814623,autobiographer
1,http://www.wikidata.org/entity/Q2919046,biochemist
2,http://www.wikidata.org/entity/Q1930187,journalist
3,http://www.wikidata.org/entity/Q15980158,non-fiction writer
4,http://www.wikidata.org/entity/Q6625963,novelist
5,http://www.wikidata.org/entity/Q12144794,prosaist
6,http://www.wikidata.org/entity/Q18844224,science fiction writer
7,http://www.wikidata.org/entity/Q3745071,science writer
8,http://www.wikidata.org/entity/Q901,scientist
9,http://www.wikidata.org/entity/Q28389,screenwriter


### Consulta 4

Asimov se casó más de una vez. Para cada uno de los matrimonios queremos conocer el nombre de la esposa y las fechas de inicio y finalización. Los resultados deben aparecer ordenados cronológicamente.

Para resolver esta consulta necesitarás acceder a los cualificadores de nodos sentencia y necesitarás entender los prefijos que usa Wikidata. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Qualifiers) te resulte útil.

In [15]:
%%wdsparql
SELECT ?wife ?startime ?endtime ?wifeLabel WHERE {
      #(P26 es el código de la propiedad cónyuge)
      wd:Q34981 p:P26 ?wifestatement. #utilizamos p para obtener los nodos de los statements de wife  y los usamos de sujeto en 
      #las siguientes tripletas para obtener lo que queremos
      ?wifestatement ps:P26 ?wife. #valor del statement (ps apunta al statement object)
      #con pq accedemos a los cualificadores de la sentencia
      ?wifestatement pq:P580 ?startime.
      ?wifestatement pq:P582 ?endtime.
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?startime)

,wife,startime,endtime,wifeLabel
0,http://www.wikidata.org/entity/Q107693884,1942-07-26T00:00:00Z,1973-01-01T00:00:00Z,Gertrude Asimov
1,http://www.wikidata.org/entity/Q517435,1973-01-01T00:00:00Z,1992-01-01T00:00:00Z,Janet Asimov


### Consulta 5

Asimov recibió muchos premios a lo largo de su carrera. Queremos obtener la lista de premios y, para cada uno de ellos, la fecha y la obra por la que fue premiado (si están disponibles). Los resultados se deben mostrar ordenador por fecha.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#OPTIONAL) te resulte útil.

Realizamos una consulta similar a la anterior utilizando los prefijos de Wikidata. Para mostrar los premios sin obra y con obra (si tiene disponibles) utilizamos OPTIONAL que permite describir subgrafos que pueden formar o no parte de la solución.

In [17]:
%%wdsparql
SELECT ?premio ?premioLabel ?fecha ?obra ?obraLabel  WHERE {
      #(P166 es el código de la propiedad award received)
      wd:Q34981 p:P166 ?premiostatement. #utilizamos p para obtener los nodos de los statements de sus premios  y los usamos de sujeto en las siguientes tripletas para obntener lo que queremos
      ?premiostatement ps:P166 ?premio. #valor del statement
      ?premiostatement pq:P585 ?fecha. #P585 point in time
      OPTIONAL {?premiostatement pq:P1686 ?obra. } #P1686 for work
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?fecha)

,premio,premioLabel,fecha,obra,obraLabel
0,http://www.wikidata.org/entity/Q188914,Hugo Award,1963-01-01T00:00:00Z,None,None
1,http://www.wikidata.org/entity/Q6143927,James T. Grady-James H. Stack Award for Interp...,1965-01-01T00:00:00Z,None,None
2,http://www.wikidata.org/entity/Q188914,Hugo Award,1966-01-01T00:00:00Z,http://www.wikidata.org/entity/Q20899148,Foundation Trilogy
3,http://www.wikidata.org/entity/Q987744,Edward E. Smith Memorial Award,1967-01-01T00:00:00Z,None,None
4,http://www.wikidata.org/entity/Q266012,Nebula Award for Best Novel,1972-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
5,http://www.wikidata.org/entity/Q255032,Hugo Award for Best Novel,1973-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
6,http://www.wikidata.org/entity/Q595998,Locus Award for Best Novel,1973-01-01T00:00:00Z,http://www.wikidata.org/entity/Q389750,The Gods Themselves
7,http://www.wikidata.org/entity/Q3174253,Klumpke-Roberts Award,1975-01-01T00:00:00Z,None,None
8,http://www.wikidata.org/entity/Q936683,Nebula Award for Best Novelette,1976-01-01T00:00:00Z,http://www.wikidata.org/entity/Q468132,The Bicentennial Man
9,http://www.wikidata.org/entity/Q1056265,Hugo Award for Best Novelette,1977-01-01T00:00:00Z,http://www.wikidata.org/entity/Q468132,The Bicentennial Man


### Consulta 6

Ahora queremos conocer todas las obras escritas por Asimov del tipo [obras literarias de ciencia ficción (Q3238422)](https://www.wikidata.org/wiki/Q3238422) y cualquiera de sus subgéneros. Los resultados se deben mostrar ordenados por nombre.

No pasa nada si en el resultado aparecen mezclados novelas, relatos, colecciones...

Buscamos en la primera tripleta las obras que escribio Asimov, a continuación sus generos y por último vemos cuales de esos generos son subclases de obras literarias de ciencia ficción.

In [18]:
%%wdsparql
SELECT ?obra ?obraLabel WHERE {
      #(P50 es el código de la propiedad autor)
      ?obra wdt:P50  wd:Q34981 . #todas las obras escritas por Asimov
      ?obra wdt:P136 ?genero. #generos de las obras P136
      ?genero wdt:P279* wd:Q3238422. #subclase (* para propagar)
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?obraLabel)

,obra,obraLabel
0,http://www.wikidata.org/entity/Q603705,. . . That Thou Art Mindful of Him
1,http://www.wikidata.org/entity/Q4631777,2430 A. D.
2,http://www.wikidata.org/entity/Q4657902,A Loint of Paw
3,http://www.wikidata.org/entity/Q3549799,A Statue for Father
4,http://www.wikidata.org/entity/Q3618113,Anniversary
5,http://www.wikidata.org/entity/Q2872379,Author! Author!
6,http://www.wikidata.org/entity/Q4920795,Black Friar of the Flame
7,http://www.wikidata.org/entity/Q4925104,Blank!
8,http://www.wikidata.org/entity/Q3511886,Blind Alley
9,http://www.wikidata.org/entity/Q2855170,Breeds There a Man...?


### Consulta 7

Vamos a volver a recuperar las mismas obras de la consulta anterior pero mostrando también la fecha de publicación de cada una. Las obras deben aparece aunque no tengan fecha de publicación asociada. En caso de que una obra tenga varias fechas de publicación, se mostrará sólo la más antigua. Los resultados deben aparecer en orden cronológico.

Para calcular la fecha correcta tendrás que agrupar las respuestas por obra y aplicar una función de agregación sobre las fechas de publicación. Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#Grouping) te resulte útil.

Obtenemos las obras como en la consulta anterior y utilizamos OPTIONAL para obtener las fechas de publicacion y que si no  tienen fecha asociada tenerlo también. Por otro lado, como las obras pueden tener más de una fecha de publicación, agrupamos las respuestas por obra y obtenemos el mínimo (MIN) de las fechas de cada obra.

In [20]:
%%wdsparql
SELECT ?obraLabel (MIN(?fecha) AS ?fecha_mas_antigua) WHERE {
      #(P50 es el código de la propiedad autor)
      ?obra wdt:P50  wd:Q34981 . #todas las obras escritas por Asimov
      ?obra wdt:P136 ?genero. #generos de las obras P136
      ?genero wdt:P279* wd:Q3238422. #subclase (* para propagar)
      OPTIONAL {?obra wdt:P577 ?fecha.} #fecha de publicacion P577
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY(?obraLabel)
ORDER BY ASC(?fecha_mas_antigua)

,obraLabel,fecha_mas_antigua
0,Foundation series,None
1,Lucky Starr series,None
2,Satisfaction Guaranteed,None
3,Foundation Trilogy,None
4,The Weapon Too Dreadful to Use,1939-01-01T00:00:00Z
5,Trends,1939-01-01T00:00:00Z
6,Marooned off Vesta,1939-03-01T00:00:00Z
7,Half-Breeds on Venus,1940-01-01T00:00:00Z
8,Ring Around the Sun,1940-01-01T00:00:00Z
9,The Callistan Menace,1940-01-01T00:00:00Z


### Consulta 8

Asimov es sin duda un autor prolífico. ¿Cuántas obras escribió a lo largo de su vida? Vamos a considerar sólo instancias directas de [literary work (Q7725634)](https://www.wikidata.org/wiki/Q7725634).

Al igual que en la consulta 6, buscamos las obras de Asimov cuyo genero sea subclase de literary work y seleccionamos COUNT de las obras para obtener el número de obras.

In [19]:
%%wdsparql
SELECT (COUNT (?obra) as ?numero_obras) WHERE {
      #(P50 es el código de la propiedad autor)
      ?obra wdt:P50  wd:Q34981 . #todas las obras escritas por Asimov
      ?obra wdt:P136 ?genero. #generos de las obras P136
      ?genero wdt:P279* wd:Q7725634. #subclase de obra literaria (* para propagar)
}

,numero_obras
0,204


### Consulta 9

De todas las obras que escribió a lo largo de su vida, queremos recuperar los nombres y fechas de publicación de aquellas escritas entre 1970 y 1980. Si alguna obra tiene varias fechas de publicación, se mostrará si alguna de ellas está dentro de ese periodo.

Puede que [esto](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial#FILTER) te resulte útil.

In [21]:
%%wdsparql
SELECT ?obraLabel ?fecha WHERE {
      #(P50 es el código de la propiedad autor)
      ?obra wdt:P50  wd:Q34981 . #todas las obras escritas por Asimov
      ?obra wdt:P577 ?fecha. #recuperamos solo las que tienen una fecha asociada
      FILTER("1970-01-01"^^xsd:dateTime <= ?fecha && ?fecha < "1980-01-01"^^xsd:dateTime).
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY (?fecha)

,obraLabel,fecha
0,Waterclap,1970-01-01T00:00:00Z
1,2430 A. D.,1970-01-01T00:00:00Z
2,Asimov's Guide to Shakespeare,1970-01-01T00:00:00Z
3,The Solar System and Back,1970-01-19T00:00:00Z
4,As Chemist to Chemist,1970-05-01T00:00:00Z
5,The Stars in their Courses,1971-01-01T00:00:00Z
6,Q3345921,1972-01-01T00:00:00Z
7,The Gods Themselves,1972-01-01T00:00:00Z
8,Mirror Image,1972-01-01T00:00:00Z
9,The Early Asimov,1972-01-01T00:00:00Z


### Consulta 10

¿Qué otros escritores de ciencia ficción estudiaron en algún centro donde también estudió Asimov? Para cada uno de ellos muestra su nombre y fechas de nacimiento y defunción (si están disponibles) y el centro donde estudió. Muestra los resultados ordenados alfabéticamente por centro y luego por autor.

In [22]:
%%wdsparql
SELECT ?centro_educativoLabel ?estudiantesLabel ?fechas_nacimiento ?fechas_muerte WHERE {
      #(P69 es el código de la propiedad educated at)
      wd:Q34981 wdt:P69 ?centro_educativo . #centros educativos en los que estuvo Asimov
      ?estudiantes wdt:P69 ?centro_educativo. #estudiantes que han estudiado en los centros
      ?estudiantes wdt:P106 wd:Q18844224. #P106 codigo de ocupacion y Q18844224 codigo de escritor de ciencia ficcion
      ?estudiantes wdt:P569 ?fechas_nacimiento. 
      OPTIONAL {?estudiantes wdt:P570 ?fechas_muerte. }
   SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ASC(?centro_educativoLabel) ASC(?estudiantesLabel)


,centro_educativoLabel,estudiantesLabel,fechas_nacimiento,fechas_muerte
0,Boys and Girls High School,Isaac Asimov,1920-01-02T00:00:00Z,1992-04-06T00:00:00Z
1,Columbia University,Alaya Dawn Johnson,1982-03-31T00:00:00Z,None
2,Columbia University,Algis Budrys,1931-01-09T00:00:00Z,2008-06-09T00:00:00Z
3,Columbia University,Arthur W. Saha,1923-10-31T00:00:00Z,1999-11-19T00:00:00Z
4,Columbia University,Ben Parris,1961-01-01T00:00:00Z,None
5,Columbia University,David G. Hartwell,1941-07-10T00:00:00Z,2016-01-20T00:00:00Z
6,Columbia University,David Kyle,1919-02-14T00:00:00Z,2016-09-18T00:00:00Z
7,Columbia University,Donald Barr,1921-08-08T00:00:00Z,2004-02-05T00:00:00Z
8,Columbia University,Doris Pitkin Buck,1898-01-03T00:00:00Z,1980-12-04T00:00:00Z
9,Columbia University,Dorothy B. Hughes,1904-08-10T00:00:00Z,1993-05-06T00:00:00Z


__Fecha de las consultas: 29 de abril de 2022__